In [ ]:
import pandas as pd
df = pd.read_csv('/content/intakes.csv')

In [52]:
# Drop rows with NaN values in the 'Phone' column
df = df.dropna(subset=['Phone'])

# Convert phone numbers from floats to integers and remove the leading '1' if present
intakePhoneList = df['Phone'].apply(lambda x: f"+{int(str(int(x)))}").tolist()

In [54]:
intakePhoneList = list(set(intakePhoneList))

In [55]:
len(intakePhoneList)

19493

In [56]:
df['Phone'].nunique()

19493

In [65]:
import requests as req
import json
import time

api_token = '09f0c9f071a1efe5c7417f4c8f9ec34ef7cf5b59a7f6618a48dda739c18f974ccb499d388106d81eae2cc47b40ee507c4e9f58cba32ca9e63e541da7ee544ff3551d8fbad7f42cfce60336c1c637f1fc45b77d2099cbd96b0f03fb8b29407a1a4e3830e89dbbac1347f5d305539525a6453f1aac'
accountId = "RA9cf203d9431444088f53c869b881f4ed"

url = f'https://api.ringba.com/v2/{accountId}/calllogs'
headers = {
    'Authorization': f'Token {api_token}',
    'Content-Type': 'application/json'
}

listCallIds = []

for phoneNumber in intakePhoneList:
  payload = {
      "reportStart": "2023-12-01T00:00:00Z",
      "reportEnd": "2024-08-19T09:00:00Z",
      "filters": [
          {
            "anyConditionToMatch": [
              {
                "column": "inboundPhoneNumber",
                "value": phoneNumber,
                "comparisonType": "EQUALS"
              }
            ]
          },
      ],
      "orderByColumns": [
          {
              "column": "callDt",
              "direction": "desc"
          }
      ],
      "size": 100,
  }

  res = req.post(url, headers=headers, data=json.dumps(payload))

  if res.status_code == 200:
        print(f"Request for {phoneNumber} was successful!")
        call_logs = res.json()

        # Collect only the top inboundCallId
        if call_logs["report"]["records"]:
            top_inboundCallId = call_logs["report"]["records"][0]["inboundCallId"]
            listCallIds.append(top_inboundCallId)
            print(f"Top inboundCallId for {phoneNumber}: {top_inboundCallId}")
        else:
            print(f"No records found for {phoneNumber}.")
  else:
      print(f"Request for {phoneNumber} failed with status code {res.status_code}")
      print(res.text)  # Print the error message

  time.sleep(0.3)

Streaming output truncated to the last 5000 lines.
Request for +13164612013 was successful!
Top inboundCallId for +13164612013: RGB81AEBF8BDEB1795D34EC97408CF768C326F82ECFRGB3602
Request for +18433384289 was successful!
No records found for +18433384289.
Request for +14072832050 was successful!
No records found for +14072832050.
Request for +12529550403 was successful!
No records found for +12529550403.
Request for +14059533265 was successful!
No records found for +14059533265.
Request for +18623108210 was successful!
No records found for +18623108210.
Request for +15126961394 was successful!
No records found for +15126961394.
Request for +13368641489 was successful!
No records found for +13368641489.
Request for +12562219542 was successful!
No records found for +12562219542.
Request for +17576461863 was successful!
No records found for +17576461863.
Request for +12707909851 was successful!
No records found for +12707909851.
Request for +12565539691 was successful!
Top inboundCallId fo

In [67]:
 # dictionary of lists
dict = {'id': listCallIds}

df = pd.DataFrame(dict)
df.to_csv('ringba_ids.csv')

In [68]:
len(listCallIds)

2803

In [41]:
print(json.dumps(call_logs, indent=4))

{
    "isSuccessful": true,
    "transactionId": "TR453df469-0b44-4f7d-9ddc-ae595ad406ff",
    "report": {
        "partialResult": false,
        "records": [
            {
                "campaignName": "Your Best Leads | SSDI | Calls",
                "publisherName": "Your Best Leads",
                "targetName": "SSDI | Your Best Leads",
                "targetNumber": "+18018558088",
                "buyer": "Trajector Disability",
                "campaignId": "CAbbef832a3d2e46788d33e1e0c5206082",
                "publisherId": "AFaac2cbe5e02943dd8a625f0f8e2394dc",
                "publisherSubId": "",
                "targetId": "TA9c796f1b0f2b42819aca27544929912b",
                "buyerId": "BU2693407dc5c6446bb48bc6ea527693a2",
                "inboundCallId": "RGB4338076243AA7CE62727F56D89DE54F5111B8016RGB1B05",
                "callDt": 1716928881234,
                "inboundPhoneNumber": "+18703354395",
                "number": "+18887172471",
                "numberId

In [ ]:
webhook_url = f"https://webhook.ringba.com/hook/WH521228c1ccce4567a6e00932b34f17e5?revenue=&intake=yes&callUUID={inbound_call_id}"
res = req.get(webhook_url)

In [2]:
import pandas as pd
df = pd.read_csv('/content/ringba_ids.csv')

In [4]:
listItems = df['id'].tolist()

In [10]:
import requests as req
import time

badList = []
# Loop through listCallIds and send a GET request to the webhook
for inbound_call_id in listItems:
    webhook_url = f"https://webhook.ringba.com/hook/WH521228c1ccce4567a6e00932b34f17e5?revenue=&intake=yes&callUUID={inbound_call_id}"
    res = req.get(webhook_url)

    if res.status_code == 200:
        print(f"Successfully sent webhook for Call UUID: {inbound_call_id}")
    else:
        print(f"Failed to send webhook for Call UUID: {inbound_call_id} with status code {res.status_code}")
        badList.append(inbound_call_id)

    # Wait for 0.1 seconds before the next webhook call
    time.sleep(0.3)

Failed to send webhook for Call UUID: RGBE1389B791BA9E463881721B67B8EA9B586D2429FV3O1501 with status code 404
Failed to send webhook for Call UUID: RGBF2996A424F1EC38CA4E58A62501A1739AB391667V38YN01 with status code 404
Failed to send webhook for Call UUID: RGBDC459C8030B0151B411448909D7FE28CC9A183D0RGB8F02 with status code 404
Failed to send webhook for Call UUID: RGBAFFEB0DB062397FA235B7C556BA9AA8FB7E679D4V348901 with status code 404
Failed to send webhook for Call UUID: RGB16CFE5A8C30B8BBC2D48CB318B95234B3193A952RGBD703 with status code 404
Failed to send webhook for Call UUID: RGB102D77C680D6EE8C55728591B5EB2981B1BF50F9V348501 with status code 404
Failed to send webhook for Call UUID: RGBBC9C3F4149B14E4D9F7A11695050390E28277D16RGBBC02 with status code 404
Failed to send webhook for Call UUID: RGB3EF5061A3F715CB86780159FC2130A70C859B149V3RTU01 with status code 404
Successfully sent webhook for Call UUID: RGBBF2EEE3E217ACD46427ACE40B6D254DAE6100B27RGB2202
Successfully sent webhook fo

In [12]:
len(badList)

2237

In [13]:
print(2803 - 2237)

566


In [14]:
 # dictionary of lists
dict = {'id': badList}

df2 = pd.DataFrame(dict)
df2.to_csv('bad_ringba_ids.csv')

In [9]:
res.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1"/>\r\n<title>404 - File or directory not found.</title>\r\n<style type="text/css">\r\n<!--\r\nbody{margin:0;font-size:.7em;font-family:Verdana, Arial, Helvetica, sans-serif;background:#EEEEEE;}\r\nfieldset{padding:0 15px 10px 15px;} \r\nh1{font-size:2.4em;margin:0;color:#FFF;}\r\nh2{font-size:1.7em;margin:0;color:#CC0000;} \r\nh3{font-size:1.2em;margin:10px 0 0 0;color:#000000;} \r\n#header{width:96%;margin:0 0 0 0;padding:6px 2% 6px 2%;font-family:"trebuchet MS", Verdana, sans-serif;color:#FFF;\r\nbackground-color:#555555;}\r\n#content{margin:0 0 0 2%;position:relative;}\r\n.content-container{background:#FFF;width:96%;margin-top:8px;padding:10px;position:relative;}\r\n-->\r\n</style>\r\n</head>\r\n<body>\r\n<div id="header"><h1>Server Error</h1></